# Baseline

### Data preprocessing

```
27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30@ds-0003-inv.jpg
27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30@ds-0003-aughor-inv.jpg
27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30@ds-0003-aughor.jpg
27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30@ds-0003.jpg
```

In [4]:
import os
import random
import shutil
from collections import defaultdict

In [46]:
def collect_files(base_dirs):
    file_dict = defaultdict(list)
    for base_dir in base_dirs:
        for root, dirs, files in os.walk(base_dir):
            for file in files:
                if file.endswith(('.jpg', '.txt')):
                    base_name = "-".join(file[:-4].split('-')[:2])  # Убираем аугментации
                    file_dict[base_name].append(os.path.join(root, file))
    return file_dict

def split_dataset(plane_files, helicopter_files, dataset_size, train_ratio=0.8, valid_ratio=0.1):
   # Подготовка путей для dataset
   train_dir = 'datasets/main_dataset/train'
   valid_dir = 'datasets/main_dataset/valid'
   test_dir = 'datasets/main_dataset/test'
   
   # Создаём директории
   for dir_path in [train_dir, valid_dir, test_dir]:
       os.makedirs(os.path.join(dir_path, 'images'), exist_ok=True)
       os.makedirs(os.path.join(dir_path, 'labels'), exist_ok=True)
   
   # Подсчитаем сколько нужно изображений для каждой части
   num_train = int(dataset_size * train_ratio)
   num_valid = int(dataset_size * valid_ratio)
   num_test = dataset_size - num_train - num_valid

   print(num_train, num_valid, num_test)
    
   plane_keys = list(plane_files.keys())
   random.shuffle(plane_keys)  # Перемешиваем список

   helicopter_keys = list(helicopter_files.keys())
   random.shuffle(helicopter_keys)  # Перемешиваем список

   train_set, valid_set, test_set = set(), set(), set()
    
   for i in range(num_train):
       if i%2 == 0:
           train_set.update(plane_files[plane_keys.pop()])
       else:
           train_set.update(helicopter_files[helicopter_keys.pop()])
           
   for i in range(num_valid):
       if i%2 == 0:
           valid_set.update(plane_files[plane_keys.pop()])
       else:
           valid_set.update(helicopter_files[helicopter_keys.pop()])

   for _ in range(num_test):
       if i%2 == 0:
           test_set.update(plane_files[plane_keys.pop()])
       else:
           test_set.update(helicopter_files[helicopter_keys.pop()])

   return train_set, valid_set, test_set

def copy_files(file_set, dest_dir):
   for file in file_set:
       sub_dir = 'images' if file.endswith('.jpg') else 'labels'
       dest_path = os.path.join(dest_dir, sub_dir, os.path.basename(file))
       shutil.copy(file, dest_path)

In [48]:
dataset_size = 5_000

# Сбор данных для каждого класса
plane_files = collect_files(['datasets/dataset/photos/0-plane/labels',
                            'datasets/dataset/photos/0-plane/images'])

helicopter_files = collect_files(['datasets/dataset/photos/1-helicopter/labels',
                            'datasets/dataset/photos/1-helicopter/images'])
print(len(plane_files), len(helicopter_files))
# Разбиваем на train/valid/test
train_set, valid_set, test_set = split_dataset(plane_files, helicopter_files, dataset_size)
print(len(train_set), len(valid_set), len(test_set))

#Копируем файлы в соответствующие директории
copy_files(train_set, 'datasets/main_dataset/train')
copy_files(valid_set, 'datasets/main_dataset/valid')
copy_files(test_set, 'datasets/main_dataset/test')

15403 4990
4000 500 500
32000 4000 4000


In [45]:
for key in plane_files.keys():
    print(key)
    [print(e) for e in plane_files[key]]
    break

14052021_(t3)@ds-1524
datasets/dataset/photos/0-plane/labels/14052021_(t3)@ds-1524-aughor-inv.txt
datasets/dataset/photos/0-plane/labels/14052021_(t3)@ds-1524-aughor.txt
datasets/dataset/photos/0-plane/labels/14052021_(t3)@ds-1524-inv.txt
datasets/dataset/photos/0-plane/labels/14052021_(t3)@ds-1524.txt
datasets/dataset/photos/0-plane/images/14052021_(t3)@ds-1524-aughor-inv.jpg
datasets/dataset/photos/0-plane/images/14052021_(t3)@ds-1524-inv.jpg
datasets/dataset/photos/0-plane/images/14052021_(t3)@ds-1524-aughor.jpg
datasets/dataset/photos/0-plane/images/14052021_(t3)@ds-1524.jpg


In [49]:
def collect_backgrounds(background_dir):
    backgrounds = []
    for root, dirs, files in os.walk(background_dir):
        for file in files:
            if file.endswith('.jpg'):
                backgrounds.append(os.path.join(root, file))
    return backgrounds

def split_backgrounds(backgrounds, train_ratio=0.8, valid_ratio=0.1):
    random.shuffle(backgrounds)
    num_background_train = int(len(backgrounds) * train_ratio)
    num_background_valid = int(len(backgrounds) * valid_ratio)
    
    background_train = backgrounds[:num_background_train]
    background_valid = backgrounds[num_background_train:num_background_train + num_background_valid]
    background_test = backgrounds[num_background_train + num_background_valid:]
    
    return background_train, background_valid, background_test

def copy_backgrounds(background_set, dest_dir):
    for background in background_set:
        dest_path = os.path.join(dest_dir, 'images', os.path.basename(background))
        shutil.copy(background, dest_path)

In [50]:
backgrounds = collect_backgrounds('datasets/dataset/photos/backgrounds/images')

# Разбиваем background снимки на train/valid/test
background_train, background_valid, background_test = split_backgrounds(backgrounds)

# Копируем background изображения в существующие директории
copy_backgrounds(background_train, 'datasets/main_dataset/train')
copy_backgrounds(background_valid, 'datasets/main_dataset/valid')
copy_backgrounds(background_test, 'datasets/main_dataset/test')

In [52]:
len(backgrounds)

216

In [54]:
def count_files(directory):
    return len([file for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))])

# Папки, которые нужно проверить
directories = {
    'train': ('datasets/main_dataset/train/images', 'datasets/main_dataset/train/labels'),
    'valid': ('datasets/main_dataset/valid/images', 'datasets/main_dataset/valid/labels'),
    'test': ('datasets/main_dataset/test/images', 'datasets/main_dataset/test/labels')
}

for key, (img_dir, label_dir) in directories.items():
    img_count = count_files(img_dir)
    label_count = count_files(label_dir)
    print(f"{key.capitalize()}:")
    print(f"  Количество файлов в images: {img_count}")
    print(f"  Количество файлов в labels: {label_count}")

Train:
  Количество файлов в images: 16172
  Количество файлов в labels: 16000
Valid:
  Количество файлов в images: 2021
  Количество файлов в labels: 2000
Test:
  Количество файлов в images: 2023
  Количество файлов в labels: 2000


## Training

In [56]:
from ultralytics import YOLO

In [57]:
model = YOLO("yolo11s.pt")
model.train(data="data.yaml", epochs=100, batch=-1, plots=True, save=True,
           lr0= 0.00746,
            lrf= 0.00942,
            momentum= 0.8946,
            weight_decay= 0.00058,
            warmup_epochs= 2.92744,
            warmup_momentum= 0.71774,
            box= 8.4543,
            cls= 0.48317,
            dfl= 1.32973,
            hsv_h= 0.0161,
            hsv_s= 0.9,
            hsv_v= 0.60851,
            degrees= 0.0,
            translate= 0.12099,
            scale= 0.53349,
            shear= 0.0,
            perspective= 0.0,
            flipud= 0.0,
            fliplr= 0.41098,
            bgr= 0.0,
            mosaic= 0.91628,
            mixup= 0.0,
            copy_paste= 0.0,
)

Ultralytics 8.3.10 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train68, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /home/user1/datasets/baseline_dataset/train/labels.cache... 3591 images, 172 backgro
val: Scanning /home/user1/datasets/baseline_dataset/valid/labels.cache... 447 images, 21 backgrounds


Plotting labels to runs/detect/train68/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.00746' and 'momentum=0.8946' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0006978125), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train68
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      19.3G      1.935      10.58     0.8195        101        640: 100%|██████████| 49/4
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|█████


                   all        468        447       0.69      0.374      0.484      0.162

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     48/100      19.4G      1.057     0.4271     0.7355         91        640: 100%|██████████| 49/4
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|█████

                   all        468        447      0.993       0.99      0.995      0.753



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


KeyboardInterrupt: 